In [1]:
import pandas as pd
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

import MySQLdb 
import numpy as np

In [2]:
ieo = pd.DataFrame()
price = pd.DataFrame()
grades = pd.DataFrame()


conn = MySQLdb.connect(host='tokenmetrics-restored-27-05.cxuzrhvtziar.us-east-1.rds.amazonaws.com', user='admin', passwd='WiG8Rled2cTvZ5JibJui',db='tokenmetrics')
cursor = conn.cursor()

query = '''SELECT * FROM icos;'''
data = pd.read_sql_query(query, conn)
ieo = pd.concat([ieo,data])

query = 'SELECT * FROM ico_price_daily_summaries where currency ="USD";'
data = pd.read_sql_query(query, conn)
price = pd.concat([price,data])

query = 'SELECT * FROM ico_ml_grade_history;'
data = pd.read_sql_query(query, conn)
grades = pd.concat([grades,data])
    
conn.close() 

In [3]:
ieo.sort_values('ico_market_cap', ascending=False, inplace=True)
ieo = ieo.drop(ieo[ieo['id'] == 3245].index)
ieo = ieo[:150]
ieo['weight'] = ieo['ico_market_cap'] / ieo['ico_market_cap'].sum()
ieo = pd.merge(ieo,price[['ico_id','ico_symbol']], left_on='id',right_on='ico_id',how='inner')
ieo = ieo.drop_duplicates('id')
weight = ieo[['ico_symbol','weight']]
weight = weight.sort_values('ico_symbol')

ieos = ieo['id'][:150]
df = price[price['ico_id'].isin(ieos)]
df = df.drop_duplicates(subset=['ico_symbol','date']).sort_values(by='date')


#temp = temp.dropna()

In [4]:
temp = df['close'].groupby([df['date'],df['ico_symbol']]).mean()
temp = temp.unstack().sort_index()
temp.index = pd.to_datetime(temp.index)

In [5]:
# Calculate expected returns and sample covariance
mu = expected_returns.mean_historical_return(temp.loc['2019-01-01':])
S = risk_models.sample_cov(temp.loc['2019-01-01':])

# Optimise for maximal Sharpe ratio
ef = EfficientFrontier(mu, S, weight_bounds=(0, 0.1))
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
ef.save_weights_to_file("weights.csv")  # saves to file
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

OrderedDict([('ABBC', 0.0), ('ADA', 0.0), ('AE', 0.0), ('ALGO', 0.0), ('AMPL', 0.02233), ('AR', 0.05207), ('ATOM', 0.0), ('BAT', 0.0), ('BAY', 0.00418), ('BCD', 0.0), ('BCH', 0.0), ('BCN', 0.0), ('BNB', 0.0), ('BNT', 0.0), ('BOTX', 0.09367), ('BSV', 0.0), ('BTC', 0.0), ('BTG', 0.0), ('BTM', 0.0), ('BTRS', 0.00077), ('BTS', 0.0), ('BTT', 0.0), ('BZT', 0.0), ('CCC', 0.00294), ('CCXX', 0.07066), ('CEL', 0.02064), ('CENNZ', 0.0), ('CKB', 0.05176), ('COMP', 0.01299), ('CRO', 0.0), ('DAI', 0.0), ('DASH', 0.0), ('DCR', 0.0), ('DEV', 0.02413), ('DGB', 0.0), ('DOGE', 0.01387), ('EKT', 0.0), ('ELA', 0.0), ('ELAMA', 0.0), ('ENJ', 0.0), ('EON', 0.0), ('EOS', 0.0), ('ETC', 0.0), ('ETH', 0.0), ('FST', 0.01519), ('FXC', 0.1), ('GAP', 0.0), ('GTN', 0.0), ('GUSD', 0.0), ('GXC', 0.0), ('HC', 0.0), ('HDAC', 0.02774), ('HEDG', 0.0), ('HIVE', 0.0), ('HMR', 0.0), ('HOT', 0.0), ('HT', 0.0), ('ICX', 0.0), ('INB', 0.0), ('INO', 0.0), ('IOST', 0.0), ('KBC', 0.0), ('KCS', 0.0), ('KIN', 0.0), ('KMD', 0.0), ('KNC'

C:\Users\Aagam\Anaconda3\lib\site-packages\pypfopt\risk_models.py:68: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  "The covariance matrix is non positive semidefinite. Amending eigenvalues."
C:\Users\Aagam\Anaconda3\lib\site-packages\pypfopt\risk_models.py:87: UserWarning: Could not fix matrix. Please try a different risk model.
  warnings.warn("Could not fix matrix. Please try a different risk model.")


(2.004568604751039, 0.1588257180687363, 12.495259765752552)

In [9]:
weights = pd.DataFrame.from_dict(dict(cleaned_weights), orient='index')
weights = weights[weights[0] > 0]

price = price.sort_values('date',ascending=False)
price = price[price['date'] == price['date'].iloc[0]]

allocation = pd.DataFrame.from_dict(allocation,orient='index')
allocation.columns = ['Token Volume']
allocation['ico_symbol'] = allocation.index

allocation = pd.merge(allocation, price[['close','ico_symbol']])

allocation['Holding Amount'] = abs(allocation['Token Volume']) * allocation['close']
allocation['Position'] = np.where(allocation['Token Volume']<0, 'Short', 'Long')
allocation

In [10]:
allocation = pd.DataFrame()
allocation['ico_symbol'] = weights.index.tolist()
allocation['weights'] = weights[0].tolist()

In [20]:
close = []
for symbol in allocation['ico_symbol'].tolist():
    temp = price[price['ico_symbol'] == symbol]
    temp = temp.sort_values('date',ascending=False)
    temp = temp[temp['date'] == temp['date'].iloc[0]]
    close.append(temp['close'].values[0])

In [22]:
allocation['price'] = close

In [26]:
allocation['Holding Amount'] = allocation['weights'] * 10000

In [27]:
allocation['% Holding'] = allocation['Holding Amount'] / allocation['Holding Amount'].sum()

In [28]:
allocation

,ico_symbol,weights,price,Holding Amount,% Holding
0,AMPL,0.02233,1.866241,223.3,0.02233
1,AR,0.05207,1.534375,520.7,0.05207
2,BAY,0.00418,0.175920,41.8,0.00418
3,BOTX,0.09367,0.064931,936.7,0.09367
4,BTRS,0.00077,82.910934,7.7,0.00077
5,CCC,0.00294,0.001729,29.4,0.00294
6,CCXX,0.07066,20.431122,706.6,0.07066
7,CEL,0.02064,0.431572,206.4,0.02064
8,CKB,0.05176,0.005493,517.6,0.05176
9,COMP,0.01299,157.967163,129.9,0.01299


### Views

In [29]:
from pypfopt import black_litterman
from pypfopt.black_litterman import BlackLittermanModel
from pypfopt.efficient_frontier import EfficientFrontier

grades = grades[grades['ico_id'].isin(ieos)]
grades = pd.merge(grades,price[['ico_id','ico_symbol']], on='ico_id',how='inner')
grades = grades.drop_duplicates(['ico_id','date'])
grades = grades.sort_values('date', ascending=False)
grades = grades[grades['date'] == grades['date'].iloc[0]]

In [33]:
grades['view'] = grades['final_grade'] / grades['final_grade'].sum()
views = grades[['ico_symbol','view']]
views.set_index('ico_symbol', inplace=True)
views = views.to_dict()
viewdict = views['view']

temp = df['close'].groupby([df['date'],df['ico_symbol']]).mean()
temp = temp.unstack().sort_index()
temp.index = pd.to_datetime(temp.index)

cov_matrix = temp.loc['2019-01-01':].cov()

In [34]:
bl = BlackLittermanModel(cov_matrix, pi="equal", absolute_views=viewdict, omega='default')

rets = bl.bl_returns()
ef = EfficientFrontier(rets, cov_matrix, weight_bounds=(0, 0.1))

# OR use return-implied weights
delta = black_litterman.market_implied_risk_aversion(temp.loc['2019-01-01':])
bl.bl_weights(delta.values)
weights = bl.clean_weights()

C:\Users\Aagam\Anaconda3\lib\site-packages\pypfopt\black_litterman.py:442: RuntimeWarning: invalid value encountered in multiply
  A = risk_aversion * self.cov_matrix
C:\Users\Aagam\Anaconda3\lib\site-packages\pypfopt\base_optimizer.py:87: RuntimeWarning: invalid value encountered in less
  clean_weights[np.abs(clean_weights) < cutoff] = 0


In [37]:
weights = pd.DataFrame.from_dict(dict(weights), orient='index')
weights = weights[weights[0] > 0]

In [38]:
weights

,0


In [12]:
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=10000,short_ratio=0.0001)
allocation, leftover = da.lp_portfolio()
print("Discrete allocation:", allocation)
print("Funds remaining: ${:.2f}".format(leftover))

Discrete allocation: {'HT': 5.0, 'TRX': 555171.0, 'XLM': 1106.0, 'XTZ': 28.0, 'LBC': -5.0, 'XRP': -4.0}
Funds remaining: $0.08


In [13]:
price = price.sort_values('date',ascending=False)
price = price[price['date'] == price['date'].iloc[0]]

allocation = pd.DataFrame.from_dict(allocation,orient='index')
allocation.columns = ['Token Volume']
allocation['ico_symbol'] = allocation.index

allocation = pd.merge(allocation, price[['close','ico_symbol']])

allocation['Holding Amount'] = abs(allocation['Token Volume']) * allocation['close']
allocation['Position'] = np.where(allocation['Token Volume']<0, 'Short', 'Long')
allocation

,Token Volume,ico_symbol,close,Holding Amount,Position
0,5.0,HT,4.238911,21.194553,Long
1,555171.0,TRX,0.017635,9790.570328,Long
2,1106.0,XLM,0.096894,107.164875,Long
3,28.0,XTZ,2.895190,81.065320,Long
4,-5.0,LBC,0.025488,0.127438,Short
5,-4.0,XRP,0.199965,0.799861,Short
